In [ ]:
import json
import pandas as pd
import os
from collections import defaultdict

In [ ]:
alphas = [10, 1, 0.1, 0.01]
experiment_path = './data/results/CIFAR100/alpha_{}/logs'
metrics = [
    'server_accuracy',
    'server_recall',
    'server_precision',
]

#round_threshold_one = 600
#round_threshold_two = 1000
#accuracy_threshold_one = 0.84
#accuracy_threshold_two = 0.86

round_threshold_one = 600
round_threshold_two = 800
accuracy_threshold_one = 0.3
accuracy_threshold_two = 0.4

In [ ]:
def read_json(path):
    with open(path) as json_file:
        return json.load(json_file)
    
def moving_average(data, length = 10):
    moving_average_data = []
    for i, x in enumerate(data):
        if i < length:
            moving_average_data.append(sum(data[0:(i+1)]) / (i + 1))
        else:
            moving_average_data.append(sum(data[(i+1-length):(i+1)]) / length)
    return moving_average_data

In [ ]:
data = defaultdict(dict)
for alpha in alphas:
    alpha_key = str(alpha).replace('.','')
    temp_path = experiment_path.format(alpha_key)
    algorithms = os.listdir(temp_path)
    for alg in algorithms:
        data[alpha_key][alg] = read_json(os.path.join(temp_path, alg))

In [ ]:
def find_threshold(average_metric, threshold):
    metric_list = [i for (i, x) in enumerate(average_data) if x >= threshold]
    if len(metric_list) > 0:
        return metric_list[0]
    else:
        return '-'

In [ ]:
round_threshold_one_data = []
round_threshold_two_data = []
accuracy_threshold_one_data = []
accuracy_threshold_two_data = []
metric_name = []
alg_name = []

r1 = 'Round {}'.format(round_threshold_one)
r2 = 'Round {}'.format(round_threshold_two)
a1 = '{}%'.format(int(accuracy_threshold_one * 100))
a2 = '{}%'.format(int(accuracy_threshold_two * 100))

table_dict = defaultdict(list)

for alpha in alphas:
    alpha_key = str(alpha).replace('.', '')
    
    for metric in metrics:
        for alg in data[alpha_key]:
            alg_name.append(alg)
            metric_name.append(metric)
            metric_data = data[alpha_key][alg][metric]
            average_data = moving_average(metric_data)
            
            table_dict[r1].append('{:.3f}'.format(average_data[round_threshold_one - 1]))
            table_dict[r2].append('{:.3f}'.format(average_data[round_threshold_two - 1]))
            table_dict[a1].append(find_threshold(average_data, accuracy_threshold_one))
            table_dict[a2].append(find_threshold(average_data, accuracy_threshold_two))
            table_dict['alpha'].append(alpha)
            table_dict['Algorithm'].append(alg)
            table_dict['Metric'].append(metric)

In [ ]:
df = pd.DataFrame(table_dict)

In [ ]:
df = df.set_index(['Metric', 'Algorithm'])
df = df.pivot(columns = 'alpha')
df = df.reset_index()
df = df.set_index(['Metric', 'Algorithm'])
df.head(10)

In [ ]:
print(df.to_latex())